In [1]:
import geopandas as gpd

# Load data and convert to metric system (S-JTSK)
gdf = gpd.read_file("Městské_části.geojson")
gdf = gdf.to_crs("EPSG:5514")

# Calculate area in km2
gdf['area_km2'] = gdf.geometry.area / 1e6

# Rename columns to standard names
gdf = gdf.rename(columns={
    'nazev_mc': 'district_name',
    'kod_mc': 'district_id'
})

# Keep only relevant columns and sort
gdf = gdf[['district_name', 'district_id', 'area_km2', 'geometry']]
gdf = gdf.sort_values('district_name').reset_index(drop=True)

# Export
gdf.to_file("prague_districts_mest.geojson", driver="GeoJSON")
print(f"Saved {len(gdf)} districts.")
gdf.head()

Saved 57 districts.


,district_name,district_id,area_km2,geometry
0,Praha 1,500054,5.538429,"MULTIPOLYGON (((-743408.143 -1042037.04, -7434..."
1,Praha 10,500224,18.599771,"MULTIPOLYGON (((-736946.983 -1043572.498, -736..."
2,Praha 11,547034,9.793681,"MULTIPOLYGON (((-738808.792 -1048377.459, -738..."
3,Praha 12,547107,23.317910,"MULTIPOLYGON (((-742423.432 -1050304.67, -7424..."
4,Praha 13,539694,13.196803,"MULTIPOLYGON (((-750136.923 -1044504.521, -750..."


In [2]:
import geopandas as gpd
import osmnx as ox

# Define tags for nightlife and tourism
tags = {
    'amenity': ['bar', 'pub', 'nightclub', 'restaurant', 'cafe'],
    'tourism': ['attraction', 'museum', 'hotel', 'hostel']
}

print("Downloading data...")
gdf = ox.features_from_place("Prague, Czech Republic", tags)

# Convert to meters first, then calculate centroid (correct way)
gdf = gdf.to_crs("EPSG:5514")
gdf['geometry'] = gdf.geometry.centroid

# Filter points only within Prague boundary
mask = ox.geocode_to_gdf("Prague, Czech Republic").to_crs("EPSG:5514").geometry.iloc[0]
gdf = gdf[gdf.geometry.within(mask)]

# Create a single 'type' column and cleanup
gdf['type'] = gdf['amenity'].fillna(gdf['tourism'])
gdf = gdf[['name', 'type', 'geometry']]

# Save
gdf.to_file("prague_tourism.geojson", driver="GeoJSON")
print(f"Saved {len(gdf)} points.")

Saved 5720 points.


In [3]:
import pandas as pd
import geopandas as gpd

# Load raw listings
df = pd.read_csv("listings.csv")

# Clean price column (remove $ and commas)
if df['price'].dtype == 'O':
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

# Filter: active listings only (reviewed in last 12 months)
# This removes "dead" listings that skew statistics
df = df[df['number_of_reviews_ltm'] > 0].copy()

# Feature 1: Entire home
df['is_entire_home'] = (df['room_type'] == 'Entire home/apt').astype(int)

# Feature 2: Economy vs Luxury (Based on Percentiles)
p25 = df['price'].quantile(0.25)
p75 = df['price'].quantile(0.75)

df['is_economy'] = (df['price'] <= p25).astype(int)
df['is_luxury'] = (df['price'] >= p75).astype(int)

# Create geometry
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
)

# Export only necessary columns
cols = ['id', 'price', 'is_entire_home', 'is_economy', 'is_luxury', 'geometry']
gdf[cols].to_file("listings_clean.geojson", driver="GeoJSON")

print(f"Saved {len(gdf)} active listings.")
print(f"Economy threshold: < {p25}")
print(f"Luxury threshold: > {p75}")

<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Michael\AppData\Local\Temp\ipykernel_24952\2829959221.py:9: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)


Saved 8098 active listings.
Economy threshold: < 2086.0
Luxury threshold: > 4720.75


In [4]:
import geopandas as gpd
import pandas as pd
import json
import glob
import ast
from shapely.geometry import shape

# Crime category codes
VIOLENT = [1, 3, 5, 7, 8]
BURGLARY = [18, 19, 20]
THEFT = [35, 41, 43]
DISORDER = [76, 111]

# Load all geojson files manually (to fix list-column issues)
files = glob.glob("2024*.geojson")
data_list = []

print(f"Merging {len(files)} files...")

for f in files:
    with open(f, 'r', encoding='utf-8') as file:
        raw = json.load(file)

    # Extract attributes and geometry
    df_attr = pd.json_normalize([feat['properties'] for feat in raw['features']])
    geoms = [shape(feat['geometry']) for feat in raw['features']]

    temp_gdf = gpd.GeoDataFrame(df_attr, geometry=geoms, crs="EPSG:4326")
    data_list.append(temp_gdf)

gdf = pd.concat(data_list, ignore_index=True)

# Filter valid crimes (Relevance 4 = confirmed crime scene)
gdf = gdf[(gdf['relevance'] == 4) & (gdf['state'].isin([1, 2]))].copy()

# Helper to map codes to names
def get_category(val):
    # Parse string to list if necessary
    if isinstance(val, str):
        try: val = ast.literal_eval(val)
        except: val = []

    # Check codes
    for code in (val if isinstance(val, list) else []):
        if code in VIOLENT: return "Violent"
        if code in BURGLARY: return "Burglary"
        if code in THEFT: return "Theft"
        if code in DISORDER: return "Disorder"
    return "Other"

gdf['category'] = gdf['types'].apply(get_category)

# Save
gdf[['category', 'geometry']].to_file("crime.geojson", driver="GeoJSON")
print(f"Saved {len(gdf)} crimes. Summary:")
print(gdf['category'].value_counts())

Merging 12 files...
Saved 1282560 crimes. Summary:
category
Other       1138851
Disorder      76601
Theft         34819
Burglary      21611
Violent       10678
Name: count, dtype: int64


In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# --- 1. LOAD BASE DATA ---
print("Loading maps and stats...")

# Load districts (metric system)
gdf = gpd.read_file("prague_districts_mest.geojson").to_crs("EPSG:5514")

# Load population data
df_socio = pd.read_csv("population.csv", sep=";")

# Load transport stops
df_mhd = pd.read_csv("transport_stops.csv").rename(columns={'public_transport_count': 'mhd_count'})

# Fix naming inconsistencies
def clean_name(name):
    if pd.isna(name): return ""
    return str(name).replace("\xa0", " ").replace(" - ", "-").replace("–", "-").strip()

gdf['district_name'] = gdf['district_name'].apply(clean_name)
df_socio['district_name'] = df_socio['district_name'].apply(clean_name)
if 'district_name' in df_mhd.columns:
    df_mhd['district_name'] = df_mhd['district_name'].apply(clean_name)

# Merge base tables
gdf = gdf.merge(df_socio, on='district_name', how='left')
gdf = gdf.merge(df_mhd, on='district_name', how='left')

# Calc area & distance to center
center = Point(-743000, -1043000) # Old Town Square
gdf['area_km2'] = gdf.geometry.area / 1e6
gdf['dist_center_km'] = gdf.geometry.centroid.distance(center) / 1000


# --- 2. PROCESS SPATIAL DATA ---
print("Processing spatial joins...")

def get_counts(filename):
    try:
        points = gpd.read_file(filename).to_crs("EPSG:5514")
        return gpd.sjoin(points, gdf[['district_name', 'geometry']], predicate="within")
    except:
        return pd.DataFrame()

# A) Airbnb
airbnb = get_counts("listings_clean.geojson")
stats_ab = airbnb.groupby('district_name').agg({
    'id': 'count', 'price': 'mean', 'is_economy': 'sum', 'is_luxury': 'sum', 'is_entire_home': 'sum'
}).reset_index().rename(columns={'id': 'airbnb_count', 'price': 'price_avg', 'is_economy': 'airbnb_economy', 'is_luxury': 'airbnb_luxury', 'is_entire_home': 'airbnb_entire_home'})

# B) Crime
crime = get_counts("crime.geojson")
stats_cr_total = crime.groupby('district_name').size().reset_index(name='crime_total')
# Pivot for crime types
stats_cr_types = pd.crosstab(crime['district_name'], crime['category']).reset_index()
stats_cr_types.columns = ['district_name'] + ['crime_' + c for c in stats_cr_types.columns if c != 'district_name']

# C) Tourism / Nightlife (Filtered)
poi = get_counts("prague_tourism.geojson")
# Filter irrelevant types
ignore = ['fountain', 'dormitory', 'library', 'monastery', 'place_of_worship', 'planetarium', 'shelter', 'clock']
poi = poi[~poi['type'].isin(ignore)]

stats_poi_total = poi.groupby('district_name').size().reset_index(name='poi_count')
# Pivot for POI types
stats_poi_types = pd.crosstab(poi['district_name'], poi['type']).reset_index()
stats_poi_types.columns = ['district_name'] + ['poi_' + c for c in stats_poi_types.columns if c != 'district_name']


# --- 3. MERGE & CLEANUP ---
print("Merging results...")

tables = [stats_ab, stats_cr_total, stats_cr_types, stats_poi_total, stats_poi_types]
for table in tables:
    gdf = gdf.merge(table, on='district_name', how='left')

# Fill NaNs with 0 (except price)
cols_fill = ['airbnb_count', 'mhd_count']
cols_fill += [c for c in gdf.columns if c.startswith('crime_') or c.startswith('poi_')]
gdf[cols_fill] = gdf[cols_fill].fillna(0)


# --- 4. CALCULATE METRICS ---
print("Feature engineering...")

# Target variable
gdf['crime_rate'] = (gdf['crime_total'] / gdf['population']) * 1000

# Densities (per km2)
gdf['airbnb_density'] = gdf['airbnb_count'] / gdf['area_km2']
gdf['airbnb_luxury_density'] = gdf['airbnb_luxury'] / gdf['area_km2']
gdf['mhd_density'] = gdf['mhd_count'] / gdf['area_km2']
gdf['poi_density'] = gdf['poi_count'] / gdf['area_km2']

# Ratios (per capita)
gdf['university_ratio'] = gdf['people_university'] / gdf['population']
gdf['foreigner_ratio'] = gdf['foreigners'] / gdf['population']
gdf['tourist_intensity'] = gdf['nights_non_residents'] / gdf['population']


# --- 5. EXPORT ---
# Organize columns
cols_cr = [c for c in gdf.columns if c.startswith('crime_') and c not in ['crime_total', 'crime_rate']]
cols_poi = [c for c in gdf.columns if c.startswith('poi_') and c != 'poi_count']

final_cols = [
    'district_name', 'population', 'area_km2', 'dist_center_km',
    'crime_total', 'crime_rate',
] + cols_cr + [
    'airbnb_count', 'airbnb_density',
    'airbnb_economy', 'airbnb_luxury', 'airbnb_luxury_density', 'airbnb_entire_home',
    'price_avg',
    'poi_count', 'poi_density'
] + cols_poi + [
    'nights_non_residents', 'tourist_intensity',
    'foreigners', 'foreigner_ratio',
    'people_university', 'university_ratio',
    'unemployment',
    'mhd_count', 'mhd_density'
]

gdf[final_cols].to_csv("final_dataset_mest.csv", index=False)
print("Done! Saved to final_dataset_mest.csv")
print(gdf[final_cols].head(3))

Loading maps and stats...
Processing spatial joins...
Merging results...
Feature engineering...
Done! Saved to final_dataset_mest.csv
  district_name  population   area_km2  dist_center_km  crime_total  \
0       Praha 1       28921   5.538429        0.362492        14712   
1      Praha 10      119110  18.599771        5.767105        10055   
2      Praha 11       78410   9.793681        8.809829         9581   

   crime_rate  crime_Burglary  crime_Disorder  crime_Other  crime_Theft  ...  \
0  508.696103             336            1085        11335         1781  ...   
1   84.417765             631             518         7949          896  ...   
2  122.191047             203             233         8694          421  ...   

   poi_density  nights_non_residents  tourist_intensity  foreigners  \
0   302.432331               7023738         242.859445        8277   
1    15.967939                523750           4.397196       28455   
2     8.576959                301749           

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --- 1. LOAD DATA ---
# Load dataset and map
df = pd.read_csv("final_dataset_mest.csv")
gdf_map = gpd.read_file("prague_districts_mest.geojson").to_crs("EPSG:5514")

# Merge data with geometry
gdf = gdf_map.merge(df, on='district_name', how='left')

cols_to_drop = [c for c in gdf.columns if c.endswith('_y')]
gdf = gdf.drop(columns=cols_to_drop)
gdf.columns = [c.replace('_x', '') for c in gdf.columns]

# --- 2. DEFINE MAPPING (57 -> 22) ---
# Mapping small cadastral districts to administrative districts
mapping = {
    'Praha 1': 'Praha 1',
    'Praha 2': 'Praha 2',
    'Praha 3': 'Praha 3',
    'Praha 4': 'Praha 4', 'Praha-Kunratice': 'Praha 4',
    'Praha 5': 'Praha 5', 'Praha-Slivenec': 'Praha 5',
    'Praha 6': 'Praha 6', 'Praha-Lysolaje': 'Praha 6', 'Praha-Nebušice': 'Praha 6',
    'Praha-Přední Kopanina': 'Praha 6', 'Praha-Suchdol': 'Praha 6',
    'Praha 7': 'Praha 7', 'Praha-Troja': 'Praha 7',
    'Praha 8': 'Praha 8', 'Praha-Březiněves': 'Praha 8',
    'Praha-Dolní Chabry': 'Praha 8', 'Praha-Ďáblice': 'Praha 8',
    'Praha 9': 'Praha 9',
    'Praha 10': 'Praha 10',
    'Praha 11': 'Praha 11', 'Praha-Křeslice': 'Praha 11',
    'Praha-Šeberov': 'Praha 11', 'Praha-Újezd': 'Praha 11',
    'Praha 12': 'Praha 12', 'Praha-Libuš': 'Praha 12',
    'Praha 13': 'Praha 13', 'Praha-Řeporyje': 'Praha 13',
    'Praha 14': 'Praha 14', 'Praha-Dolní Počernice': 'Praha 14',
    'Praha 15': 'Praha 15', 'Praha-Dolní Měcholupy': 'Praha 15',
    'Praha-Dubeč': 'Praha 15', 'Praha-Petrovice': 'Praha 15', 'Praha-Štěrboholy': 'Praha 15',
    'Praha 16': 'Praha 16', 'Praha-Lipence': 'Praha 16',
    'Praha-Lochkov': 'Praha 16', 'Praha-Velká Chuchle': 'Praha 16', 'Praha-Zbraslav': 'Praha 16',
    'Praha 17': 'Praha 17', 'Praha-Zličín': 'Praha 17',
    'Praha 18': 'Praha 18', 'Praha-Čakovice': 'Praha 18',
    'Praha 19': 'Praha 19', 'Praha-Satalice': 'Praha 19', 'Praha-Vinoř': 'Praha 19',
    'Praha 20': 'Praha 20',
    'Praha 21': 'Praha 21', 'Praha-Běchovice': 'Praha 21',
    'Praha-Klánovice': 'Praha 21', 'Praha-Koloděje': 'Praha 21',
    'Praha 22': 'Praha 22', 'Praha-Benice': 'Praha 22',
    'Praha-Kolovraty': 'Praha 22', 'Praha-Královice': 'Praha 22', 'Praha-Nedvězí': 'Praha 22'
}

gdf['admin_district'] = gdf['district_name'].map(mapping)
gdf['admin_district'] = gdf['admin_district'].fillna(gdf['district_name'])

# --- 3. PREPARE WEIGHTED DATA ---
print("Calculating weighted values...")

# Prepare for weighted average: Value * Weight (Population)
# This prevents small villages from skewing the average of a large district
gdf['unemp_weighted'] = gdf['unemployment'] * gdf['population']
gdf['price_weighted'] = gdf['price_avg'] * gdf['population']

# --- 4. AGGREGATION ---
print("Aggregating statistics...")

# Dissolve merges geometries and sums the data
gdf_large = gdf.dissolve(by='admin_district', aggfunc='sum').reset_index()

# --- 5. RECALCULATE RATES ---
print("Recalculating rates and averages...")

gdf_large['area_km2'] = gdf_large.geometry.area / 10**6

# 1. Weighted Averages
gdf_large['unemployment'] = gdf_large['unemp_weighted'] / gdf_large['population']
gdf_large['price_avg'] = gdf_large['price_weighted'] / gdf_large['population']

# 2. Rates
gdf_large['crime_rate'] = (gdf_large['crime_total'] / gdf_large['population']) * 1000

# 3. Densities
gdf_large['airbnb_density'] = gdf_large['airbnb_count'] / gdf_large['area_km2']
gdf_large['mhd_density'] = gdf_large['mhd_count'] / gdf_large['area_km2']
gdf_large['poi_density'] = gdf_large['poi_count'] / gdf_large['area_km2']

if 'airbnb_luxury' in gdf_large.columns:
    gdf_large['airbnb_luxury_density'] = gdf_large['airbnb_luxury'] / gdf_large['area_km2']

# 4. Ratios
gdf_large['university_ratio'] = gdf_large['people_university'] / gdf_large['population']
gdf_large['foreigner_ratio'] = gdf_large['foreigners'] / gdf_large['population']
gdf_large['tourist_intensity'] = gdf_large['nights_non_residents'] / gdf_large['population']

# 5. Distance to center
center_point = Point(-743000, -1043000)
gdf_large['dist_center_km'] = gdf_large.geometry.centroid.distance(center_point) / 1000

# Cleanup
gdf_large = gdf_large.drop(columns=['unemp_weighted', 'price_weighted'])

# --- 6. EXPORT ---
# Rename for consistency
gdf_large = gdf_large.rename(columns={'admin_district': 'district_name'})

# Save
gdf_large.to_csv("final_dataset_obvod.csv", index=False)

print(f"Done! Aggregated to {len(gdf_large)} districts.")
print(gdf_large[['district_name', 'population', 'unemployment']].head())

Calculating weighted values...
Aggregating statistics...
Recalculating rates and averages...
Done! Aggregated to 22 districts.
  district_name                                   district_name  population  \
0       Praha 1                                         Praha 1       28921   
1      Praha 10                                        Praha 10      119110   
2      Praha 11  Praha 11Praha-KřeslicePraha-ÚjezdPraha-Šeberov       86775   
3      Praha 12                             Praha 12Praha-Libuš       70893   
4      Praha 13                          Praha 13Praha-Řeporyje       72892   

   unemployment  
0      2.250000  
1      2.430000  
2      2.164435  
3      2.393900  
4      1.951725  


In [10]:
import pandas as pd

# 1. LOAD DATA
df_small = pd.read_csv("final_dataset_mest.csv")   # 57 districts
df_large = pd.read_csv("final_dataset_obvod.csv")  # 22 districts

# 2. REMOVE DUPLICATE COLUMNS
# Sometimes merging creates 'col.1', we need to remove those
def clean_columns(df):
    cols_to_drop = [c for c in df.columns if '.1' in c]
    if cols_to_drop:
        print(f"Dropping duplicate columns: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)
    return df

df_small = clean_columns(df_small)
df_large = clean_columns(df_large)

# 3. VERIFY SUMS (Consistency Check)
# Totals (population, counts) must match exactly between datasets
check_cols = ['population', 'airbnb_count', 'airbnb_luxury', 'crime_total']

print("\n--- SUM COMPARISON ---")
print(f"{'Metric':<20} | {'Small (57)':<15} | {'Large (22)':<15} | {'Diff'}")
print("-" * 65)

for col in check_cols:
    sum_small = df_small[col].sum()
    sum_large = df_large[col].sum()
    diff = sum_small - sum_large
    print(f"{col:<20} | {sum_small:<15.0f} | {sum_large:<15.0f} | {diff}")

# 4. VERIFY WEIGHTED AVERAGES
# Unemployment must be averaged by population, not simple mean
print("\n--- WEIGHTED AVERAGE CHECK (Unemployment) ---")

def calc_weighted_avg(df, val_col, weight_col):
    return (df[val_col] * df[weight_col]).sum() / df[weight_col].sum()

avg_small = calc_weighted_avg(df_small, 'unemployment', 'population')
avg_large = calc_weighted_avg(df_large, 'unemployment', 'population')

print(f"Weighted Avg (Small): {avg_small:.4f}%")
print(f"Weighted Avg (Large): {avg_large:.4f}%")

if abs(avg_small - avg_large) < 0.001:
    print("SUCCESS: Weighted averages match!")
else:
    print("WARNING: Averages do not match.")

# 5. EXPORT CLEAN FILES
df_small.to_csv("final_dataset_mest.csv", index=False)
df_large.to_csv("final_dataset_obvod.csv", index=False)
print("\nSaved clean datasets.")

Dropping duplicate columns: ['poi_density.1']
Dropping duplicate columns: ['district_name.1', 'poi_density.1']

--- SUM COMPARISON ---
Metric               | Small (57)      | Large (22)      | Diff
-----------------------------------------------------------------
population           | 1397880         | 1397880         | 0
airbnb_count         | 8098            | 8098            | 0.0
airbnb_luxury        | 1827            | 1827            | 0.0
crime_total          | 167386          | 167386          | 0

--- WEIGHTED AVERAGE CHECK (Unemployment) ---
Weighted Avg (Small): 2.4012%
Weighted Avg (Large): 2.4012%
SUCCESS: Weighted averages match!

Saved clean datasets.
